In [4]:
import torch
import numpy as np
import pandas as pd

In [2]:
old_dataset = torch.load('/mnt/yzy/NIMCGCN/new_dataset.pt')

In [3]:
def get_edge(matrix):
    edge_x = []
    edge_y = []
    for i in range(0,matrix.shape[0]):
        for j in range(0,matrix.shape[1]):
            if(matrix[i][j] != 0):
                edge_x.append(i)
                edge_y.append(j)
    
    edge = torch.tensor([edge_x,edge_y])
    print(edge.shape)
    return edge
get_edge(old_dataset['mm']['data'])

torch.Size([2, 948255])


tensor([[   0,    0,    0,  ..., 1042, 1042, 1042],
        [   0,    1,    2,  ..., 1040, 1041, 1042]])

In [4]:
print(old_dataset['mm']['edge_index'].shape)

torch.Size([2, 948255])


In [5]:
old_dataset = torch.load("/mnt/yzy/NIMCGCN/new_dataset.pt")
print(old_dataset.keys())
print(old_dataset['md_p'].shape)
print(old_dataset['md_true'].shape)
print(old_dataset['mm']['data'].shape)
print(old_dataset['mm']['edge_index'].shape)
print(old_dataset['dd']['data'].shape)
print(old_dataset['dd']['edge_index'].shape)
print(old_dataset['fold_one_index'].shape)
print(old_dataset['fold_zero_index'].shape)
print(1043*1043,2166*2166)

dict_keys(['md_p', 'md_true', 'mm', 'dd', 'fold_one_index', 'fold_zero_index'])
torch.Size([1043, 2166])
torch.Size([1043, 2166])
torch.Size([1043, 1043])
torch.Size([2, 948255])
torch.Size([2166, 2166])
torch.Size([2, 4670244])
torch.Size([10, 2, 143])
torch.Size([10, 2, 225771])
1087849 4691556


In [6]:
drug_mol2vec_sim = np.loadtxt("/mnt/yzy/NIMCGCN/MultiView/drug_mol2vec_sim.csv",delimiter=',')
rna_seq_sim = np.loadtxt("/mnt/yzy/NIMCGCN/MultiView/rna_seq_sim.csv",delimiter=',')

print(drug_mol2vec_sim.shape)
print(rna_seq_sim.shape)

(2166, 2166)
(1043, 1043)


In [7]:
drug_mol2vec_sim_edge = get_edge(drug_mol2vec_sim)
drug_mol2vec_sim_data = torch.tensor(drug_mol2vec_sim)

print(drug_mol2vec_sim_edge.shape)
print(drug_mol2vec_sim_edge[0])
print(drug_mol2vec_sim_data.shape)
print(drug_mol2vec_sim_data[0])


rna_seq_sim_edge = get_edge(rna_seq_sim)
rna_seq_sim_data = torch.tensor(rna_seq_sim)

print(rna_seq_sim_edge.shape)
print(rna_seq_sim_edge[0])
print(rna_seq_sim_data.shape)
print(rna_seq_sim_data[0])

torch.Size([2, 4691554])
torch.Size([2, 4691554])
tensor([   0,    0,    0,  ..., 2165, 2165, 2165])
torch.Size([2166, 2166])
tensor([0.5190, 0.4533, 0.4735,  ..., 0.3160, 0.4666, 0.6221],
       dtype=torch.float64)
torch.Size([2, 659344])
torch.Size([2, 659344])
tensor([   2,    2,    2,  ..., 1041, 1041, 1041])
torch.Size([1043, 1043])
tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)


In [13]:
multiview_dataset = {
    'md':old_dataset['md_p'].double(),
    'mm_func':{
        'attr':old_dataset['mm']['data'].double(),
        'edge':old_dataset['mm']['edge_index']
    },
    'mm_seq':{
        'attr':rna_seq_sim_data.double(),
        'edge':rna_seq_sim_edge
    },
    'dd_seq':{
        'attr':old_dataset['dd']['data'].double(),
        'edge':old_dataset['dd']['edge_index']
    },
    'dd_mol':{
        'attr':drug_mol2vec_sim_data.double(),
        'edge':drug_mol2vec_sim_edge
    },
    'fold_one_index':old_dataset['fold_one_index'],
    'fold_zero_index':old_dataset['fold_zero_index']
}

torch.save(multiview_dataset,"/mnt/yzy/NIMCGCN/MultiView/multiview_dataset.pt")

In [2]:
dataset = torch.load("/mnt/yzy/NIMCGCN/MultiView/multiview_dataset.pt")

for i in dataset.keys():
    if(i == 'md' or i == 'fold_one_index' or i == 'fold_zero_index'):
        print(i,dataset[i].shape, type(dataset[i]),dataset[i].dtype)
    else:
        print(i,dataset[i]['attr'].shape,type(dataset[i]['attr']),dataset[i]['attr'].dtype)
        print(i,dataset[i]['edge'].shape,type(dataset[i]['edge']),dataset[i]['edge'].dtype)

md torch.Size([1043, 2166]) <class 'torch.Tensor'> torch.float64
mm_func torch.Size([1043, 1043]) <class 'torch.Tensor'> torch.float64
mm_func torch.Size([2, 948255]) <class 'torch.Tensor'> torch.int64
mm_seq torch.Size([1043, 1043]) <class 'torch.Tensor'> torch.float64
mm_seq torch.Size([2, 659344]) <class 'torch.Tensor'> torch.int64
dd_seq torch.Size([2166, 2166]) <class 'torch.Tensor'> torch.float64
dd_seq torch.Size([2, 4670244]) <class 'torch.Tensor'> torch.int64
dd_mol torch.Size([2166, 2166]) <class 'torch.Tensor'> torch.float64
dd_mol torch.Size([2, 4691554]) <class 'torch.Tensor'> torch.int64
fold_one_index torch.Size([10, 2, 143]) <class 'torch.Tensor'> torch.int64
fold_zero_index torch.Size([10, 2, 225771]) <class 'torch.Tensor'> torch.int64


In [3]:
multiview_dataset = {
    'md':dataset['md'].float(),
    'mm_func':{
        'attr':dataset['mm_func']['attr'].float(),
        'edge':dataset['mm_func']['edge']
    },
    'mm_seq':{
        'attr':dataset['mm_seq']['attr'].float(),
        'edge':dataset['mm_seq']['edge']
    },
    'dd_seq':{
        'attr':dataset['dd_seq']['attr'].float(),
        'edge':dataset['dd_seq']['edge']
    },
    'dd_mol':{
        'attr':dataset['dd_mol']['attr'].float(),
        'edge':dataset['dd_mol']['edge']
    },
    'fold_one_index':dataset['fold_one_index'],
    'fold_zero_index':dataset['fold_zero_index']
}

In [5]:
dataset = multiview_dataset
for i in dataset.keys():
    if(i == 'md' or i == 'fold_one_index' or i == 'fold_zero_index'):
        print(i,dataset[i].shape, type(dataset[i]),dataset[i].dtype)
    else:
        print(i,dataset[i]['attr'].shape,type(dataset[i]['attr']),dataset[i]['attr'].dtype)
        print(i,dataset[i]['edge'].shape,type(dataset[i]['edge']),dataset[i]['edge'].dtype)

torch.save(multiview_dataset,"/mnt/yzy/NIMCGCN/MultiView/multiview_dataset.pt")

md torch.Size([1043, 2166]) <class 'torch.Tensor'> torch.float32
mm_func torch.Size([1043, 1043]) <class 'torch.Tensor'> torch.float32
mm_func torch.Size([2, 948255]) <class 'torch.Tensor'> torch.int64
mm_seq torch.Size([1043, 1043]) <class 'torch.Tensor'> torch.float32
mm_seq torch.Size([2, 659344]) <class 'torch.Tensor'> torch.int64
dd_seq torch.Size([2166, 2166]) <class 'torch.Tensor'> torch.float32
dd_seq torch.Size([2, 4670244]) <class 'torch.Tensor'> torch.int64
dd_mol torch.Size([2166, 2166]) <class 'torch.Tensor'> torch.float32
dd_mol torch.Size([2, 4691554]) <class 'torch.Tensor'> torch.int64
fold_one_index torch.Size([10, 2, 143]) <class 'torch.Tensor'> torch.int64
fold_zero_index torch.Size([10, 2, 225771]) <class 'torch.Tensor'> torch.int64


In [3]:
dataset = torch.load("/mnt/yzy/NIMCGCN/MultiView/multiview_dataset.pt")
dataset.keys()

dict_keys(['md', 'mm_func', 'mm_seq', 'dd_seq', 'dd_mol', 'fold_one_index', 'fold_zero_index'])

In [12]:
dataset['fold_one_index'].shape

torch.Size([10, 2, 143])

In [9]:
new_md = pd.read_csv("/mnt/yzy/NIMCGCN/datasets/data(MDA108)/mirna_drug_1043_2166_1502.txt",sep=',',header=0,index_col=0)
new_md = new_md.values
new_md.sum()

1502

In [10]:
new_md_tensor = torch.tensor(new_md)
new_md_tensor.shape

torch.Size([1043, 2166])

In [14]:
new_one_index = [[],[]]
new_zero_index = [[],[]]

In [15]:
for i in range(new_md.shape[0]):
    for j in range(new_md.shape[1]):
        if(new_md[i][j] != 1):
            new_one_index[0].append(i)
            new_one_index[1].append(j)
        else:
            new_zero_index[0].append(i)
            new_zero_index[1].append(j)

In [16]:
new_one_index = torch.tensor(new_one_index)
new_zero_index = torch.tensor(new_zero_index)
print(new_one_index.shape)
print(new_zero_index.shape)

torch.Size([2, 2257636])
torch.Size([2, 1502])


In [27]:
# 使用randperm函数生成一个长度为2257636的随机排列的索引序列
idx = torch.randperm(new_one_index.size(1))
shuffled_one_data = new_one_index.index_select(0, idx)

idx = torch.randperm(new_zero_index.size(1))
shuffled_zero_data = new_zero_index.index_select(0, idx)

RuntimeError: [enforce fail at CPUAllocator.cpp:68] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 40775362467968 bytes. Error code 12 (Cannot allocate memory)

In [26]:
# 创建一个形状为[2, 2257636]的张量，表示(x, y)
data = torch.tensor([[i, i+1] for i in range(5)]).T
print(data.shape)
print(data)
# 使用randperm函数生成一个长度为2257636的随机排列的索引序列
idx = torch.randperm(data.size(1))
print(idx)
shuffled_data = data.index_select(1, idx)
# 使用gather函数按照生成的随机索引对张量的第二个维度进行重排列
print(shuffled_data)

torch.Size([2, 5])
tensor([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5]])
tensor([3, 1, 0, 2, 4])
tensor([[3, 1, 0, 2, 4],
        [4, 2, 1, 3, 5]])
